In [1]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 

from langchain_core.output_parsers import StrOutputParser
import langchain
# load_dotenv(dotenv_path='.env')
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")



In [2]:
print(f"lang chain.version : {langchain.__version__}")
print(f"Open API Key : {OPENAI_API_KEY[:10]}")

lang chain.version : 0.3.27
Open API Key : gsk_yRNqBi


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# LLM 설정
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="openai/gpt-oss-120b",
    temperature=0.7
)

def create_attraction_recommendation_chain():
    """여행지에서 대표 명소를 추천하는 체인"""
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "당신은 세계 각국의 관광 명소에 대해 잘 알고 있는 여행 전문가입니다."),
        ("system", "사용자가 입력한 도시나 국가에서 가장 대표적이고 유명한 관광 명소 1가지만 추천해주세요."),
        ("system", "명소의 이름만 간단하게 답변하세요. 설명은 하지 마세요."),
        ("user", "{destination}")
    ])
    
    return prompt | llm | StrOutputParser()

def create_attraction_detail_chain():
    """명소에 대한 상세 정보를 제공하는 체인"""
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "당신은 관광 명소에 대한 상세한 정보를 제공하는 전문 가이드입니다."),
        ("system", "다음 형식으로 명소에 대한 정보를 제공해주세요:"),
        ("system", "명소: [명소 이름]"),
        ("system", "역사: [역사적 배경과 건설/형성 과정]"),
        ("system", "특징: [주요 특징과 볼거리]"),
        ("system", "방문 팁: [실용적인 방문 정보와 팁]"),
        ("user", "{attraction}")
    ])
    
    return prompt | llm | StrOutputParser()


attraction_recommendation_chain = create_attraction_recommendation_chain()
attraction_detail_chain = create_attraction_detail_chain()

print("관광 명소 추천 2단계 체인이 생성되었습니다!")


관광 명소 추천 2단계 체인이 생성되었습니다!


In [8]:
# 두 체인을 LCEL로 연결하는 통합 함수
def travel_recommendation_system(destination):
    """
    여행지 입력받아 명소 추천 후 상세 정보 제공하는 통합 시스템
    """
    print(f"=== 여행지: {destination} ===")
    print()
    
    # 1단계: 명소 추천
    print("1단계: 대표 명소 추천 중...")
    recommended_attraction = attraction_recommendation_chain.invoke({"destination": destination})
    print(f"추천 명소: {recommended_attraction}")
    print()
    
    # 2단계: 상세 정보 제공
    print("2단계: 명소 상세 정보 조회 중...")
    detailed_info = attraction_detail_chain.invoke({"attraction": recommended_attraction})
    print("상세 정보:")
    print(detailed_info)
    print()
    
    return {
        "destination": destination,
        "recommended_attraction": recommended_attraction,
        "detailed_info": detailed_info
    }

print("통합 여행 추천 시스템이 준비되었습니다!")


통합 여행 추천 시스템이 준비되었습니다!


In [9]:
# 로마 테스트 (예상 결과: 콜로세움)
print("=" * 60)
result1 = travel_recommendation_system("로마")


=== 여행지: 로마 ===

1단계: 대표 명소 추천 중...
추천 명소: 콜로세움

2단계: 명소 상세 정보 조회 중...
상세 정보:
**명소:** 콜로세움 (Colosseum, 로마 원형극장)  

**역사:**  
- **건설 시기:** 서기 72년부터 80년 사이에 로마 황제 베스파시아스와 그의 아들 티투스에 의해 건설되었습니다. 베스파시아스는 황제 네로의 화재 복구 사업의 일환으로, 대중에게 무료 오락을 제공함으로써 신임을 얻고자 했습니다.  
- **구조와 규모:** 직경 약 189 m, 높이 약 48 m에 달하는 거대한 석재 구조물로, 약 5만 명을 수용할 수 있었습니다. 80개의 입구와 복잡한 통로(“vomitoria”)를 통해 관객이 빠르게 입·퇴장할 수 있었습니다.  
- **용도:** 검투사 경기, 야생동물 사냥(베스티아리움), 해상 전투(초기에는 물을 채워 전함을 띄우는 ‘네우마키아’), 공개 처형 등 다양한 공개 오락이 열렸습니다.  
- **중세 이후 변화:** 서기 5세기경 로마 제국이 몰락한 뒤, 콜로세움은 방치되었고, 12세기부터는 성벽과 방어 시설, 그리고 교회(성 마르코 교회)로 재활용되었습니다. 14~15세기에는 건축 자재(석재, 금속) 절도꾼들이 ‘스파르타클라’라 불리는 채석장으로 사용했습니다.  
- **보존과 복원:** 18세기부터는 고대 로마 유산에 대한 관심이 높아지며 복원 작업이 진행됐고, 1990년대와 2000년대에는 국제적인 보존 프로젝트가 진행되어 현재는 UNESCO 세계문화유산(1980년)으로 등재되어 있습니다.  

**특징:**  
1. **건축 기술:** 로마식 콘크리트와 아치, 볼트(돔) 구조를 결합한 뛰어난 공학적 설계. 트라베르스(Trabeated)와 아치가 조화돼 무게를 효율적으로 분산합니다.  
2. **복합 구조:** 지하에 ‘하이포가우스(Hypogeum)’라 불리는 복층 지하실이 있어 검투사와 동물, 무대 장치가 이동·출입할 수 있었습니다. 현대에 복원된 하이포가우스는 관람객에게 개방되